In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from glob import glob
import os
import sys
import json
from typing import Dict, List
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.join(os.getcwd(), "..", "tools"))

In [ ]:
CLASSES = [
    "CAR",
    "TRUCK",
    "TRAILER",
    "BUS",
    "VAN",
    "BICYCLE",
    "MOTORCYCLE",
    "PEDESTRIAN",
    "EMERGENCY_VEHICLE",
    "OTHER",
]

CLASS_COLORS = {
    "CAR": [0, 0.8, 0.964705882],
    "TRUCK": [0.337254902, 1, 0.71372549],
    "TRAILER": [0.352941176, 1, 0.494117647],
    "VAN": [0.921568627, 0.811764706, 0.211764706],
    "MOTORCYCLE": [0.725490196, 0.643137255, 0.329411765],
    "BUS": [0.850980392, 0.541176471, 0.525490196],
    "PEDESTRIAN": [0.91372549, 0.462745098, 0.976470588],
    "BICYCLE": [0.694117647, 0.549019608, 1],
    "EMERGENCY_VEHICLE": [0.4, 0.419607843, 0.980392157],
    "OTHER": [0.780392157, 0.780392157, 0.780392157],
}

root_path = "../data/a9_temporal_no_split"

In [ ]:
def analyze(root_path: str, point_cloud_range = []) -> Dict[str, List[int]]:
    img_label_s1_folder = os.path.join(root_path, "labels_point_clouds", "s110_lidar_ouster_south")
    img_label_s1_paths = sorted(glob(os.path.join(img_label_s1_folder, "*")))
    class_distances = {cls: [] for cls in CLASSES}
    class_coords = {cls: [] for cls in CLASSES}
    for x in img_label_s1_paths:
        img_label_s1_json_path = os.path.join(x)
        json_data = None
        with open(img_label_s1_json_path, "r") as f:
            json_data = json.load(f)
            frame_idx = list(json_data["openlabel"]["frames"].keys())[0]
            frame_objects = json_data["openlabel"]["frames"][frame_idx]["objects"]

            for obj in frame_objects.values():
                obj_type = obj["object_data"]["type"]
                if "cuboid" in obj["object_data"]:
                    loc = np.asarray(obj["object_data"]["cuboid"]["val"][:3], dtype=np.float32)

                    if len(point_cloud_range) != 0 and not (
                        loc[0] > point_cloud_range[0]
                        and loc[0] < point_cloud_range[3]
                        and loc[1] > point_cloud_range[1]
                        and loc[1] < point_cloud_range[4]
                        and loc[2] > point_cloud_range[2]
                        and loc[2] < point_cloud_range[5]
                    ):
                        continue

                    distance = np.sqrt(np.sum(np.array(loc[:2]) ** 2))
                    class_distances[obj_type].append(distance)
                    class_coords[obj_type].append(loc)
    for x in class_distances:
        class_distances[x] = np.array(class_distances[x])
        class_coords[x] = np.array(class_coords[x])
    return class_distances, class_coords

In [ ]:
def plot_class_distances(class_distances: Dict[str, List[int]], title: str = ""):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_title(title)
    ax.set_xlabel("Distance [m]")
    ax.set_ylabel("Frequency")
    for cls, distances in class_distances.items():
        ax.hist(distances, bins=50, label=cls, alpha=0.5, color=CLASS_COLORS[cls])
    ax.legend()
    plt.show()


def plot_class_histograms(class_distances: Dict[str, List[int]]):
    fig, axs = plt.subplots(5, 2, figsize=(10, 25))
    axs = axs.flatten()
    for i, (cls, distances) in enumerate(class_distances.items()):
        axs[i].hist(
            distances, bins=50, range=[0, 160], label=cls, alpha=0.5, color=CLASS_COLORS[cls]
        )
        axs[i].set_title(cls)
    plt.show()

def plot_class_hists_and_coords(class_distances: Dict[str, List[int]], class_coords: Dict[str, List[int]]):
    fig, axs = plt.subplots(10, 2, figsize=(10, 50))
    axs = axs.flatten()
    for i, (cls, distances) in enumerate(class_distances.items()):
        idx = i * 2
        axs[idx].set_title(cls)
        axs[idx].hist(
            distances, bins=50, range=[0, 160], label=cls, alpha=0.5, color=CLASS_COLORS[cls]
        )
        axs[idx+1].set_title(cls)
        axs[idx+1].set_xlim([-160, 160])
        axs[idx+1].set_ylim([-160, 160])
        axs[idx+1].scatter(class_coords[cls][:, 0], class_coords[cls][:, 1], s=0.2, label=cls, alpha=1, color=CLASS_COLORS[cls])
    plt.show()


def print_distance_stats(class_distances: Dict[str, List[int]]):
    for cls, distances in class_distances.items():
        print(
            f" {cls:<20} count: {len(distances):<10} min: {np.min(distances):<10.3f} mean: {np.mean(distances):<10.3f} max: {np.max(distances):<10.3f} std: {np.std(distances):<10.2f}"
        )
    print("=" * 100)
    print(
        f" {'Total':<20} count: {np.sum([len(x) for x in class_distances.values()]):<10} min: {np.min([np.min(x) for x in class_distances.values()]):<10.3f} mean: {np.mean([np.mean(x) for x in class_distances.values()]):<10.3f} max: {np.max([np.max(x) for x in class_distances.values()]):<10.3f} std: {np.std([np.std(x) for x in class_distances.values()]):<10.2f}"
    )

# No filtering

In [ ]:
class_distances, class_coords = analyze(root_path)

In [ ]:
print_distance_stats(class_distances)

In [ ]:
plot_class_hists_and_coords(class_distances, class_coords)

In [ ]:
plot_class_distances(class_distances, "Distance distribution for all classes")

In [ ]:
plot_class_histograms(class_distances)

# With Filtering

In [ ]:
point_cloud_range = [-25.0, -75.0, -10.0, 125.0, 75.0, 0.0]
class_distances, class_coords = analyze(root_path, point_cloud_range)

In [ ]:
print_distance_stats(class_distances)

In [ ]:
plot_class_hists_and_coords(class_distances, class_coords)

In [ ]:
plot_class_histograms(class_distances)